# Wrapping the Fine-Tuned Model into an Agentic pipeline (LangChain)

## Dependencies

### Modules 

In [ ]:
%pip install langchain

In [ ]:
%pip install langchain_community

In [ ]:
%pip install langchain-huggingface

In [ ]:
%pip install -U langchain-huggingface transformers

In [ ]:
%pip install llama_index

In [ ]:
%pip install llama-index-embeddings-huggingface

In [ ]:
%pip install langchain-openai

In [ ]:
%pip install python-dotenv

### Imports

In [1]:
from langchain.tools import tool
from langchain_openai import ChatOpenAI
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from transformers import pipeline, AutoTokenizer, AutoModelForSequenceClassification
import torch
from langchain.memory import ConversationBufferWindowMemory

from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, Settings
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core.node_parser import SentenceSplitter
from pydantic import BaseModel, Field

import os
from dotenv import load_dotenv

/Users/admin/Desktop/Spam-Jailbreak-Classification/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
use_gpu = torch.cuda.is_available()
device = 0 if use_gpu else -1

## Agent

### Loading models

In [ ]:
load_dotenv()

load your open api key

In [ ]:
open_ai_key = os.getenv("OPENAI_API_KEY") 

In [ ]:
gen_model_path = "gpt-4o-mini"
llm = ChatOpenAI(
    model=gen_model_path,
    temperature=0.2,
    api_key=open_ai_key
)

In [ ]:
classifier_path = "Fenix125/bert-spam-ham-classifier"

In [ ]:
cls_model = AutoModelForSequenceClassification.from_pretrained(classifier_path)
cls_tokenizer = AutoTokenizer.from_pretrained(classifier_path)

In [ ]:
clf = pipeline("text-classification", 
               model=cls_model, 
               tokenizer=cls_tokenizer,
               device=device
              )

In [ ]:
clf("Hello")

### Tools

#### Classifier Tool

In [ ]:
class ClassifyArgs(BaseModel):
    text: str = Field(description="text needed for classification")

@tool("classify_spam_ham", args_schema=ClassifyArgs)
def classify_spam_ham(text: str) -> str:
    """
    This tool classifies the given message as spam or ham.
    """
    pred = clf(text)[0]["label"]
    return "spam" if pred == "LABEL_1" else "ham"

#### Info retrieval

In [3]:
Settings.embed_model = HuggingFaceEmbedding("sentence-transformers/all-MiniLM-L6-v2")

In [4]:
Settings.node_parser = SentenceSplitter(chunk_size=70, chunk_overlap=10)

In [5]:
file_path = "data/student_bio.txt"

In [6]:
docs = SimpleDirectoryReader(input_files=[file_path]).load_data()
docs

[Document(id_='8f12e405-4b43-4ea5-8364-7828e55fbb54', embedding=None, metadata={'file_path': 'data/student_bio.txt', 'file_name': 'student_bio.txt', 'file_type': 'text/plain', 'file_size': 451, 'creation_date': '2025-08-15', 'last_modified_date': '2025-08-15'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, metadata_template='{key}: {value}', metadata_separator='\n', text_resource=MediaResource(embeddings=None, data=None, text='My name is Mykhailo Ivasiuk. \nI am from Zarichia, the Ivano-Frankivsk region of Ukraine. \nI was born in Ibiza, Spain. My dad is Mykhailo, and my mother is Nadiya.\nI am currently studying at the Ukrainian Catholic University in the Faculty of Applied Sciences, specializing in Computer Science.\nI am passionate about developing proble

In [7]:
index = VectorStoreIndex.from_documents(docs)

In [8]:
retriever = index.as_retriever(similarity_top_k=5)

In [9]:
type(retriever)

llama_index.core.indices.vector_store.retrievers.retriever.VectorIndexRetriever

In [16]:
nodes = retriever.retrieve("parents")
context = ".".join(n.node.get_content() for n in nodes)

In [17]:
for n in nodes:
    print(n)

Node ID: 5292ffee-9897-4410-90b8-b89feff8c527
Text: My name is Mykhailo Ivasiuk.  I am from Zarichia, the Ivano-
Frankivsk region of Ukraine.  I was born in Ibiza, Spain. My dad is
Mykhailo, and my mother is Nadiya.
Score:  0.135

Node ID: eb636851-0f37-4447-9053-339c6c48dc75
Text: I am currently studying at the Ukrainian Catholic University in
the Faculty of Applied Sciences, specializing in Computer Science. I
am passionate about developing problem-solving skills. I am currently
studying a new field for Machine Learning, which I really enjoy. I
like going to gym.
Score:  0.106



In [ ]:
context

In [ ]:
class SearchInfoArgs(BaseModel):
    query: str = Field(description="a query of user for looking down the information about Mykhailo")

@tool("search_info_about_Mykhailo_Ivasiuk", args_schema=SearchInfoArgs)
def search_info(query: str) -> str:
    """
    This tool searches for the biography of Mykhailo Ivasiuk and returns relevant information.
    Returns concatenated text of all matching documents, or an empty string if no results found.
    """
    nodes = retriever.retrieve(query)
    if not nodes:
        return "No information found."
    context = "\n\n".join(n.node.get_content() for n in nodes)
    return context

In [ ]:
tools = [classify_spam_ham, search_info]

### Implementing agent

In [ ]:
system = """
You are a helpful terminal assistant.

RULES:
- Call the "classify_spam_ham" tool if the user EXPLICITLY asks to classify spam or ham,
    OR they use trigger phrases like: "classify", "is this spam", "spam or ham",
    "label this", "check for spam", or the input starts with "Classify this:" or "Classify:".
    When you do call this tool, after it returns, output EXACTLY the tool's result
    in lowercase ("spam" or "ham"), with some extra words saying its a spam or ham
- If the user asks anything related to Mykhailo Ivasiuk's biography, for example queries like:
    "Who is Mykhailo Ivasiuk?", "Tell me about Mykhailo Ivasiuk", "What does Mykhailo study?",
    call the "search_info_about_Mykhailo_Ivasiuk" tool to with the given query to fetch his biography details,
    process them to look more natural and human, and return the result.
- If the user sends a greeting or small talk (e.g., "hello", "hi"), DO NOT call any tool.
- If a user asks a general question that doesn't match either tool, provide a neutral response.
"""

In [ ]:
prompt = ChatPromptTemplate.from_messages([
    ("system", system),
    MessagesPlaceholder("chat_history"),      
    ("human", "{input}"),                     
    MessagesPlaceholder("agent_scratchpad"),
])

In [ ]:
agent = create_tool_calling_agent(llm, tools, prompt)

In [ ]:
memory = ConversationBufferWindowMemory(
    k=20,
    return_messages=True,
    memory_key="chat_history",
    output_key="output"
)

debug

In [ ]:
from langchain.globals import set_verbose, set_debug

on = False
set_debug(on)
set_verbose(on)

In [ ]:
agent_exec = AgentExecutor(agent=agent, 
                           tools=tools,
                           memory=memory,
                           verbose=True,
                           memory_key="chat_history",
                           return_intermediate_steps=True)

In [ ]:
res = agent_exec.invoke({"input": "Hello"})
print(res["output"])

In [ ]:
res = agent_exec.invoke({"input": "Classify: hello click here"})
res

In [ ]:
res = agent_exec.invoke({"input": "Who is Michael Jordan?"})
print(res["output"])

In [ ]:
res = agent_exec.invoke({"input": "What's my name?"})
print(res["output"])

In [ ]:
res = agent_exec.invoke({"input": "my name is Bob"})
print(res["output"])

In [ ]:
res = agent_exec.invoke({"input": "What's my name?"})
print(res["output"])

In [ ]:
res = agent_exec.invoke({"input": "What's my favorite color?"})
print(res["output"])

In [ ]:
res = agent_exec.invoke({"input": "My favorite color is yellow"})
print(res["output"])

In [ ]:
res = agent_exec.invoke({"input": "What's my favorite color?"})
print(res["output"])

In [ ]:
res = agent_exec.invoke({"input": "Does Mykhailo has parents"})
print(res["output"])

In [ ]:
res